In [1]:
import pandas as pd 
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [2]:
df_train = pd.read_csv('../input/train_face_value_label.csv', dtype={' label': object, 'name': object})
lbl = LabelEncoder()
df_train['y'] = lbl.fit_transform(df_train[' label'].values)

In [3]:
# -*- coding: utf-8 -*-
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

import time, datetime
import pdb, traceback

import cv2
# import imagehash
from PIL import Image

from sklearn.model_selection import train_test_split, StratifiedKFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

In [4]:
class QRDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label=img_label
        
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        start_time = time.time()
        img = Image.open(self.img_path[index])
        
        if self.transform is not None:
            img = self.transform(img)
                
        return img, torch.from_numpy(np.array([self.img_label[index]]))
    
    def __len__(self):
        return len(self.img_path)

In [5]:
train_path = ['../input/train_data/'+x for x in df_train['name']]
train_label = df_train['y'].values

test_path = glob.glob('../input/public_test_data/*.jpg')

train_path, train_label = np.array(train_path), np.array(train_label)
test_path = np.array(test_path)

In [6]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
    
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, *meters):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = ""


    def pr2int(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


In [7]:
class VisitNet(nn.Module):
    def __init__(self):
        super(VisitNet, self).__init__()
        model = models.resnet18(True)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 256)
        self.resnet = model
        
    def forward(self, img):
        out = self.resnet(img)
        return F.log_softmax(out, dim=1)

def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, losses, top1, top5)

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, torch.max(target, 1)[0])

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(acc1[0], input.size(0))
            top5.update(acc5[0], input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        return top1

def predict(test_loader, model, tta=10):
    # switch to evaluate mode
    model.eval()
    
    test_pred_tta = None
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            end = time.time()
            for i, (input, target) in enumerate(test_loader):
                input = input.cuda()
                target = target.cuda()

                # compute output
                output = model(input)
                output = output.data.cpu().numpy()

                test_pred.append(output)
        test_pred = np.vstack(test_pred)
    
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    # data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    # top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, losses, top1)

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, torch.max(target, 1)[0])

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(acc1[0], input.size(0))
        # top5.update(acc5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            progress.pr2int(i)

In [8]:
skf = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
for flod_idx, (train_idx, val_idx) in enumerate(skf.split(train_path, train_label)):
    print(flod_idx, train_idx, val_idx)
    
    train_loader = torch.utils.data.DataLoader(
        QRDataset(train_path[train_idx], train_label[train_idx],
                transforms.Compose([
                            transforms.RandomGrayscale(),
                            # transforms.Resize((124, 124)),
                            # transforms.RandomAffine(10),
                            transforms.ColorJitter(hue=.05, saturation=.05),
                            transforms.Resize(280),
                            transforms.RandomCrop((256, 256)),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ), batch_size=50, shuffle=True, num_workers=10, pin_memory=True
    )
    
    val_loader = torch.utils.data.DataLoader(
        QRDataset(train_path[val_idx], train_label[val_idx],
                transforms.Compose([
                            # transforms.Resize((124, 124)),
                            transforms.Resize(280),
                            transforms.RandomCrop((256, 256)),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ), batch_size=10, shuffle=False, num_workers=10, pin_memory=True
    )
    
    test_loader = torch.utils.data.DataLoader(
        QRDataset(test_path, np.zeros(len(test_path)),
                transforms.Compose([
                            # transforms.Resize((124, 124)),
                            transforms.Resize(280),
                            transforms.RandomCrop((256, 256)),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ), batch_size=10, shuffle=False, num_workers=10, pin_memory=True
    )
    
    
    model = VisitNet()
    # model = nn.DataParallel(model).cuda()
    model = model.cuda()
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.Adam(model.parameters(), 0.01)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.85)
    for epoch in range(30):
        scheduler.step()
        print('Epoch: ', epoch)

        train(train_loader, model, criterion, optimizer, epoch)
        val_acc = validate(val_loader, model, criterion)
        
        torch.save(model.state_dict(), './model_dump/resnet18_fold{0}_{1}_{2}.pt'.format(flod_idx, str(epoch).zfill(2), val_acc).replace(' ', ''))

0 [ 3693  3694  3698 ... 39617 39618 39619] [   0    1    2 ... 4238 4244 4259]
Epoch:  0
[  0/714]	Time  3.859 ( 3.859)	Loss 5.8133e+00 (5.8133e+00)	Acc@1   0.00 (  0.00)
[100/714]	Time  1.849 ( 0.288)	Loss 2.9272e+00 (2.0695e+00)	Acc@1  24.00 ( 25.72)
[200/714]	Time  0.094 ( 0.262)	Loss 1.6259e+00 (1.8834e+00)	Acc@1  16.00 ( 29.29)
[300/714]	Time  0.093 ( 0.259)	Loss 1.5794e+00 (1.7069e+00)	Acc@1  46.00 ( 36.07)
[400/714]	Time  0.095 ( 0.255)	Loss 8.3924e-01 (1.5711e+00)	Acc@1  72.00 ( 41.93)
[500/714]	Time  0.355 ( 0.252)	Loss 5.1510e-01 (1.4221e+00)	Acc@1  80.00 ( 47.67)
[600/714]	Time  0.095 ( 0.252)	Loss 7.2558e-01 (1.2902e+00)	Acc@1  78.00 ( 52.96)
[700/714]	Time  0.096 ( 0.251)	Loss 4.2546e-01 (1.1758e+00)	Acc@1  80.00 ( 57.26)
 * Acc@1 85.909 Acc@5 99.899
Epoch:  1
[  0/714]	Time  2.943 ( 2.943)	Loss 2.8106e-01 (2.8106e-01)	Acc@1  86.00 ( 86.00)
[100/714]	Time  0.095 ( 0.268)	Loss 5.8633e-01 (3.6356e-01)	Acc@1  76.00 ( 87.45)
[200/714]	Time  0.093 ( 0.255)	Loss 1.4462e-01 (3.4

[600/714]	Time  0.897 ( 0.250)	Loss 1.5600e-04 (1.1546e-02)	Acc@1 100.00 ( 99.85)
[700/714]	Time  0.937 ( 0.252)	Loss 1.4976e-04 (1.0497e-02)	Acc@1 100.00 ( 99.87)
 * Acc@1 100.000 Acc@5 100.000
Epoch:  12
[  0/714]	Time  2.608 ( 2.608)	Loss 7.5469e-05 (7.5469e-05)	Acc@1 100.00 (100.00)
[100/714]	Time  0.931 ( 0.277)	Loss 8.0623e-05 (2.6638e-03)	Acc@1 100.00 ( 99.98)
[200/714]	Time  1.737 ( 0.268)	Loss 6.2690e-04 (6.0161e-03)	Acc@1 100.00 ( 99.92)
[300/714]	Time  1.080 ( 0.260)	Loss 2.3928e-03 (8.4330e-03)	Acc@1 100.00 ( 99.91)
[400/714]	Time  0.094 ( 0.252)	Loss 4.6100e-04 (9.8888e-03)	Acc@1 100.00 ( 99.89)
[500/714]	Time  0.090 ( 0.253)	Loss 1.7176e-03 (1.0075e-02)	Acc@1 100.00 ( 99.89)
[600/714]	Time  0.095 ( 0.252)	Loss 2.7989e-03 (9.4787e-03)	Acc@1 100.00 ( 99.90)
[700/714]	Time  0.094 ( 0.252)	Loss 6.3046e-04 (9.4623e-03)	Acc@1 100.00 ( 99.89)
 * Acc@1 100.000 Acc@5 100.000
Epoch:  13
[  0/714]	Time  3.042 ( 3.042)	Loss 1.1733e-04 (1.1733e-04)	Acc@1 100.00 (100.00)
[100/714]	Time

Exception in thread Thread-40:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 158, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 256, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/usr/lib/python3.6/m

KeyboardInterrupt: 

In [9]:
model.load_state_dict(torch.load('./model_dump/resnet18_fold0_11_Acc@1100.00(100.00).pt'))

test_pred = predict(test_loader, model, 10)
test_pred = np.vstack(test_pred)
test_pred = np.argmax(test_pred, 1)

In [10]:
test_pred = lbl.inverse_transform(test_pred)
test_csv = pd.DataFrame()
test_csv['name'] = [x.split('/')[-1] for x in test_path]
test_csv['label'] = test_pred
test_csv.sort_values(by='name', inplace=True)
test_csv.to_csv('tmp_newmodel_resnet18_tta10.csv', index=None, sep=',')

In [11]:
!head ../input/train_face_value_label.csv